In [2]:
import firebase_admin
from firebase_admin import credentials

cred = credentials.Certificate("../codeinterface-85b5e-firebase-adminsdk-11q7e-837ba92a03.json")
firebase_admin.initialize_app(cred)